In [ ]:
import optuna
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports loaded")

## 1. Configuration & Data Structures

In [ ]:
@dataclass
class HedgingParams:
    """Parameters to optimize for delta hedging."""
    hedge_cadence_seconds: int  # 30, 60, 300, 900
    delta_threshold: float  # 0.05 to 0.50
    hedge_ratio: float  # 0.8 to 1.0
    gamma_scalp_threshold: float  # 0.01 to 0.10
    max_hedge_per_interval: float  # Max % of position to hedge at once

@dataclass
class MarketTick:
    """Single market data point."""
    timestamp: datetime
    spot_price: float
    iv: float  # Implied volatility
    funding_rate: float

@dataclass
class OptionsPosition:
    """Simulated options position."""
    delta: float
    gamma: float
    theta: float
    vega: float
    notional: float

@dataclass
class HedgeResult:
    """Result of a hedging simulation."""
    total_pnl: float
    transaction_costs: float
    net_pnl: float
    sharpe_ratio: float
    max_drawdown: float
    hedge_count: int
    avg_delta_exposure: float

# Trading costs
MAKER_FEE = 0.0002  # 2 bps
TAKER_FEE = 0.0005  # 5 bps
SLIPPAGE = 0.0001   # 1 bp

print("✅ Data structures defined")

## 2. Synthetic Market Data Generator

In [ ]:
def generate_market_data(
    start_price: float = 50000.0,
    volatility: float = 0.6,
    duration_hours: int = 24 * 7,  # 1 week
    tick_interval_seconds: int = 10
) -> List[MarketTick]:
    """
    Generate synthetic market data with realistic crypto dynamics.
    Includes jumps, vol clustering, and funding rate cycles.
    """
    np.random.seed(42)
    
    n_ticks = (duration_hours * 3600) // tick_interval_seconds
    dt = tick_interval_seconds / (365 * 24 * 3600)  # Year fraction
    
    prices = [start_price]
    ivs = [volatility]
    funding_rates = []
    timestamps = []
    
    start_time = datetime.now() - timedelta(hours=duration_hours)
    
    for i in range(n_ticks):
        # GBM with occasional jumps
        z = np.random.standard_normal()
        
        # Add jump component (5% chance of 2-5% jump)
        if np.random.random() < 0.05:
            jump = np.random.uniform(-0.05, 0.05)
        else:
            jump = 0
        
        # Vol clustering (GARCH-like)
        vol_shock = 0.1 * (z ** 2 - 1)
        new_iv = max(0.2, min(1.5, ivs[-1] * (1 + vol_shock * 0.1)))
        
        # Price update
        ret = (0 - 0.5 * new_iv**2) * dt + new_iv * np.sqrt(dt) * z + jump
        new_price = prices[-1] * np.exp(ret)
        
        # Funding rate (8-hour cycle, typically 0.01% to 0.03%)
        hours_elapsed = i * tick_interval_seconds / 3600
        funding_cycle = np.sin(2 * np.pi * hours_elapsed / 8)
        funding = 0.0001 + 0.0002 * (1 + funding_cycle) / 2
        
        prices.append(new_price)
        ivs.append(new_iv)
        funding_rates.append(funding)
        timestamps.append(start_time + timedelta(seconds=i * tick_interval_seconds))
    
    ticks = []
    for i in range(len(timestamps)):
        ticks.append(MarketTick(
            timestamp=timestamps[i],
            spot_price=prices[i+1],
            iv=ivs[i+1],
            funding_rate=funding_rates[i]
        ))
    
    return ticks

# Generate test data
market_data = generate_market_data(duration_hours=24*7)  # 1 week
print(f"✅ Generated {len(market_data):,} market ticks")
print(f"   Price range: ${min(t.spot_price for t in market_data):,.0f} - ${max(t.spot_price for t in market_data):,.0f}")
print(f"   IV range: {min(t.iv for t in market_data):.1%} - {max(t.iv for t in market_data):.1%}")

## 3. Greeks Simulation Engine

In [ ]:
class GreeksSimulator:
    """
    Simulates how Greeks evolve as the underlying moves.
    Uses simplified Black-Scholes sensitivities.
    """
    
    def __init__(self, initial_position: OptionsPosition, strike: float, days_to_expiry: float):
        self.position = initial_position
        self.strike = strike
        self.dte = days_to_expiry
        self.initial_spot = 50000.0
        
    def update_greeks(self, spot: float, iv: float, time_passed_days: float) -> OptionsPosition:
        """
        Update Greeks based on new spot price and IV.
        Simplified model for simulation purposes.
        """
        self.dte = max(0.01, self.dte - time_passed_days)
        
        # Moneyness
        moneyness = np.log(spot / self.strike) / (iv * np.sqrt(self.dte / 365))
        
        # Delta changes with spot (simplified)
        from scipy.stats import norm
        d1 = moneyness + 0.5 * iv * np.sqrt(self.dte / 365)
        base_delta = norm.cdf(d1)
        
        # Scale by position
        position_sign = 1 if self.position.delta >= 0 else -1
        
        # Gamma peaks ATM, decays with time
        gamma = norm.pdf(d1) / (spot * iv * np.sqrt(self.dte / 365))
        gamma = gamma * abs(self.position.notional) / spot
        
        # Theta accelerates as expiry approaches
        theta = -spot * norm.pdf(d1) * iv / (2 * np.sqrt(self.dte / 365))
        theta = theta * abs(self.position.notional) / spot / 365
        
        # Vega highest ATM
        vega = spot * norm.pdf(d1) * np.sqrt(self.dte / 365)
        vega = vega * abs(self.position.notional) / spot / 100
        
        return OptionsPosition(
            delta=(base_delta - 0.5) * 2 * position_sign,  # Center around 0
            gamma=gamma * position_sign,
            theta=theta,
            vega=vega * position_sign,
            notional=self.position.notional
        )

print("✅ Greeks simulator defined")

## 4. Hedging Simulation Engine

In [ ]:
class HedgingSimulator:
    """
    Simulates hedging execution with given parameters.
    Tracks P&L, costs, and risk metrics.
    """
    
    def __init__(self, params: HedgingParams, market_data: List[MarketTick]):
        self.params = params
        self.market_data = market_data
        
    def run_simulation(self, initial_position: OptionsPosition) -> HedgeResult:
        """
        Run full simulation with given parameters.
        Returns performance metrics.
        """
        from scipy.stats import norm
        
        # Initialize
        greeks_sim = GreeksSimulator(
            initial_position=initial_position,
            strike=50000,
            days_to_expiry=30
        )
        
        hedge_position = 0.0  # In base currency (e.g., BTC)
        pnl_history = [0.0]
        total_costs = 0.0
        hedge_count = 0
        delta_exposures = []
        
        last_hedge_time = self.market_data[0].timestamp
        last_price = self.market_data[0].spot_price
        
        for i, tick in enumerate(self.market_data[1:], 1):
            # Update Greeks
            time_passed = (tick.timestamp - self.market_data[i-1].timestamp).total_seconds() / 86400
            current_greeks = greeks_sim.update_greeks(tick.spot_price, tick.iv, time_passed)
            
            # Net delta including hedge
            net_delta = current_greeks.delta + hedge_position / tick.spot_price
            delta_exposures.append(abs(net_delta))
            
            # P&L from price move (delta + gamma)
            price_change = tick.spot_price - last_price
            delta_pnl = current_greeks.delta * price_change
            gamma_pnl = 0.5 * current_greeks.gamma * price_change ** 2
            
            # Hedge P&L
            hedge_pnl = hedge_position * (tick.spot_price - last_price) / last_price
            
            # Theta decay
            theta_pnl = current_greeks.theta * time_passed
            
            tick_pnl = delta_pnl + gamma_pnl + hedge_pnl + theta_pnl
            pnl_history.append(pnl_history[-1] + tick_pnl)
            
            # Check if time to hedge
            seconds_since_hedge = (tick.timestamp - last_hedge_time).total_seconds()
            should_hedge = seconds_since_hedge >= self.params.hedge_cadence_seconds
            
            # Emergency hedge if delta exceeds threshold
            if abs(net_delta) > self.params.delta_threshold:
                should_hedge = True
            
            if should_hedge:
                # Calculate hedge amount
                target_hedge = -current_greeks.delta * self.params.hedge_ratio
                hedge_change = target_hedge - hedge_position / tick.spot_price
                
                # Limit hedge size
                max_hedge = self.params.max_hedge_per_interval * abs(current_greeks.notional) / tick.spot_price
                hedge_change = np.clip(hedge_change, -max_hedge, max_hedge)
                
                # Execute hedge with costs
                hedge_notional = abs(hedge_change) * tick.spot_price
                cost = hedge_notional * (TAKER_FEE + SLIPPAGE)
                total_costs += cost
                
                hedge_position += hedge_change * tick.spot_price
                hedge_count += 1
                last_hedge_time = tick.timestamp
            
            # Gamma scalp check
            if abs(current_greeks.gamma * price_change ** 2) > self.params.gamma_scalp_threshold * abs(current_greeks.notional):
                # Take gamma profit
                scalp_amount = 0.5 * current_greeks.gamma * price_change
                scalp_cost = abs(scalp_amount) * tick.spot_price * (TAKER_FEE + SLIPPAGE)
                total_costs += scalp_cost
            
            last_price = tick.spot_price
        
        # Calculate metrics
        pnl_array = np.array(pnl_history)
        returns = np.diff(pnl_array)
        
        total_pnl = pnl_history[-1]
        net_pnl = total_pnl - total_costs
        
        # Sharpe ratio (annualized)
        if len(returns) > 1 and np.std(returns) > 0:
            sharpe = np.mean(returns) / np.std(returns) * np.sqrt(365 * 24 * 6)  # Assuming 10-min intervals
        else:
            sharpe = 0
        
        # Max drawdown
        cummax = np.maximum.accumulate(pnl_array)
        drawdowns = pnl_array - cummax
        max_dd = np.min(drawdowns)
        
        return HedgeResult(
            total_pnl=total_pnl,
            transaction_costs=total_costs,
            net_pnl=net_pnl,
            sharpe_ratio=sharpe,
            max_drawdown=max_dd,
            hedge_count=hedge_count,
            avg_delta_exposure=np.mean(delta_exposures) if delta_exposures else 0
        )

print("✅ Hedging simulator defined")

## 5. Optuna Optimization

In [ ]:
def objective(trial: optuna.Trial) -> float:
    """
    Optuna objective function.
    Returns negative Sharpe ratio (we want to maximize it).
    """
    # Suggest parameters
    hedge_cadence = trial.suggest_categorical(
        'hedge_cadence_seconds', [30, 60, 300, 900]  # 30s, 1m, 5m, 15m
    )
    delta_threshold = trial.suggest_float('delta_threshold', 0.05, 0.50)
    hedge_ratio = trial.suggest_float('hedge_ratio', 0.80, 1.00)
    gamma_scalp_threshold = trial.suggest_float('gamma_scalp_threshold', 0.01, 0.10)
    max_hedge_per_interval = trial.suggest_float('max_hedge_per_interval', 0.10, 0.50)
    
    params = HedgingParams(
        hedge_cadence_seconds=hedge_cadence,
        delta_threshold=delta_threshold,
        hedge_ratio=hedge_ratio,
        gamma_scalp_threshold=gamma_scalp_threshold,
        max_hedge_per_interval=max_hedge_per_interval
    )
    
    # Initial position: Short gamma (typical options selling)
    initial_position = OptionsPosition(
        delta=0.15,  # Slightly long delta
        gamma=-0.02,  # Short gamma
        theta=500,  # Positive theta (collecting)
        vega=-3000,  # Short vega
        notional=1_000_000  # $1M notional
    )
    
    # Run simulation
    simulator = HedgingSimulator(params, market_data)
    result = simulator.run_simulation(initial_position)
    
    # Multi-objective: Sharpe - penalty for high drawdown and costs
    score = result.sharpe_ratio
    score -= 0.1 * abs(result.max_drawdown) / 10000  # Drawdown penalty
    score -= 0.05 * result.transaction_costs / 1000  # Cost penalty
    
    # Report intermediate values
    trial.set_user_attr('net_pnl', result.net_pnl)
    trial.set_user_attr('max_drawdown', result.max_drawdown)
    trial.set_user_attr('hedge_count', result.hedge_count)
    trial.set_user_attr('transaction_costs', result.transaction_costs)
    
    return score  # Optuna maximizes by default with direction='maximize'

print("✅ Objective function defined")

In [ ]:
# Create Optuna study
study = optuna.create_study(
    study_name='hedging_optimization',
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

print("🔄 Starting optimization (100 trials)...")
print("   This may take a few minutes.\n")

# Run optimization
study.optimize(
    objective,
    n_trials=100,
    show_progress_bar=True,
    n_jobs=1  # Single-threaded for reproducibility
)

print("\n✅ Optimization complete!")

## 6. Results Analysis

In [ ]:
# Best parameters
print("=" * 60)
print("🏆 OPTIMAL HEDGING PARAMETERS")
print("=" * 60)

best_trial = study.best_trial
print(f"\nBest Score (risk-adjusted): {best_trial.value:.4f}")
print(f"\nParameters:")
for key, value in best_trial.params.items():
    if key == 'hedge_cadence_seconds':
        cadence_map = {30: '30s', 60: '1m', 300: '5m', 900: '15m'}
        print(f"  • Hedge Cadence: {cadence_map.get(value, f'{value}s')}")
    elif 'threshold' in key or 'ratio' in key:
        print(f"  • {key.replace('_', ' ').title()}: {value:.2%}")
    else:
        print(f"  • {key.replace('_', ' ').title()}: {value:.4f}")

print(f"\nPerformance Metrics:")
print(f"  • Net P&L: ${best_trial.user_attrs['net_pnl']:,.2f}")
print(f"  • Max Drawdown: ${best_trial.user_attrs['max_drawdown']:,.2f}")
print(f"  • Hedge Count: {best_trial.user_attrs['hedge_count']:,}")
print(f"  • Transaction Costs: ${best_trial.user_attrs['transaction_costs']:,.2f}")

In [ ]:
# Parameter importance
print("\n" + "=" * 60)
print("📊 PARAMETER IMPORTANCE")
print("=" * 60)

try:
    importance = optuna.importance.get_param_importances(study)
    for param, imp in sorted(importance.items(), key=lambda x: x[1], reverse=True):
        bar = '█' * int(imp * 40)
        print(f"  {param:30s} {bar} {imp:.1%}")
except:
    print("  (Need more trials for importance analysis)")

In [ ]:
# Trials dataframe
trials_df = study.trials_dataframe()
trials_df = trials_df.sort_values('value', ascending=False)

print("\n" + "=" * 60)
print("📋 TOP 10 PARAMETER COMBINATIONS")
print("=" * 60)

display_cols = ['number', 'value', 'params_hedge_cadence_seconds', 
                'params_delta_threshold', 'params_hedge_ratio',
                'user_attrs_net_pnl', 'user_attrs_hedge_count']
display_cols = [c for c in display_cols if c in trials_df.columns]

print(trials_df[display_cols].head(10).to_string())

In [ ]:
# Visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Optimization history
ax1 = axes[0, 0]
ax1.plot(trials_df['number'], trials_df['value'], 'b-', alpha=0.5)
ax1.scatter(trials_df['number'], trials_df['value'], c=trials_df['value'], cmap='RdYlGn', s=20)
ax1.axhline(y=study.best_value, color='r', linestyle='--', label=f'Best: {study.best_value:.3f}')
ax1.set_xlabel('Trial')
ax1.set_ylabel('Score')
ax1.set_title('Optimization History')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Hedge cadence distribution
ax2 = axes[0, 1]
cadence_values = trials_df['params_hedge_cadence_seconds'].value_counts().sort_index()
colors = ['green' if v == best_trial.params['hedge_cadence_seconds'] else 'steelblue' for v in cadence_values.index]
ax2.bar([f"{v}s" for v in cadence_values.index], cadence_values.values, color=colors)
ax2.set_xlabel('Hedge Cadence')
ax2.set_ylabel('Trial Count')
ax2.set_title('Hedge Cadence Exploration')

# 3. Delta threshold vs Score
ax3 = axes[1, 0]
scatter = ax3.scatter(
    trials_df['params_delta_threshold'],
    trials_df['value'],
    c=trials_df['params_hedge_cadence_seconds'],
    cmap='viridis',
    alpha=0.7,
    s=50
)
plt.colorbar(scatter, ax=ax3, label='Cadence (s)')
ax3.axvline(x=best_trial.params['delta_threshold'], color='r', linestyle='--')
ax3.set_xlabel('Delta Threshold')
ax3.set_ylabel('Score')
ax3.set_title('Delta Threshold vs Performance')
ax3.grid(True, alpha=0.3)

# 4. P&L vs Transaction Costs
ax4 = axes[1, 1]
if 'user_attrs_net_pnl' in trials_df.columns and 'user_attrs_transaction_costs' in trials_df.columns:
    ax4.scatter(
        trials_df['user_attrs_transaction_costs'],
        trials_df['user_attrs_net_pnl'],
        c=trials_df['value'],
        cmap='RdYlGn',
        alpha=0.7,
        s=50
    )
    ax4.set_xlabel('Transaction Costs ($)')
    ax4.set_ylabel('Net P&L ($)')
    ax4.set_title('P&L vs Transaction Costs Trade-off')
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('hedging_optimization_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Results saved to hedging_optimization_results.png")

## 7. Export Recommended Configuration

In [ ]:
import json
import yaml

# Generate config
recommended_config = {
    'hedging': {
        'cadence_seconds': int(best_trial.params['hedge_cadence_seconds']),
        'delta_threshold': round(best_trial.params['delta_threshold'], 4),
        'hedge_ratio': round(best_trial.params['hedge_ratio'], 4),
        'gamma_scalp_threshold': round(best_trial.params['gamma_scalp_threshold'], 4),
        'max_hedge_per_interval': round(best_trial.params['max_hedge_per_interval'], 4),
    },
    'optimization_metadata': {
        'best_score': round(study.best_value, 4),
        'n_trials': len(study.trials),
        'optimization_date': datetime.now().isoformat(),
        'market_data_duration_hours': 24 * 7,
    },
    'expected_performance': {
        'net_pnl_usd': round(best_trial.user_attrs['net_pnl'], 2),
        'max_drawdown_usd': round(best_trial.user_attrs['max_drawdown'], 2),
        'hedge_count_per_week': int(best_trial.user_attrs['hedge_count']),
        'transaction_costs_usd': round(best_trial.user_attrs['transaction_costs'], 2),
    }
}

# Save as YAML
with open('hedging_config_optimized.yml', 'w') as f:
    yaml.dump(recommended_config, f, default_flow_style=False, sort_keys=False)

# Save as JSON
with open('hedging_config_optimized.json', 'w') as f:
    json.dump(recommended_config, f, indent=2)

print("=" * 60)
print("📁 EXPORTED CONFIGURATION FILES")
print("=" * 60)
print("\n• hedging_config_optimized.yml")
print("• hedging_config_optimized.json")
print("\n" + yaml.dump(recommended_config, default_flow_style=False, sort_keys=False))

## 8. Recommendations Summary

In [ ]:
print("\n" + "=" * 60)
print("📋 RECOMMENDATIONS FOR PRODUCTION")
print("=" * 60)

cadence = best_trial.params['hedge_cadence_seconds']
delta_thresh = best_trial.params['delta_threshold']

print(f"""
Based on optimization over 1 week of simulated market data:

1. HEDGE TIMING:
   • Use {cadence}s ({cadence//60}m) regular hedge intervals
   • Emergency hedge when |delta| > {delta_thresh:.1%}
   
2. HEDGE SIZING:
   • Hedge {best_trial.params['hedge_ratio']:.0%} of delta exposure
   • Max {best_trial.params['max_hedge_per_interval']:.0%} of position per interval
   
3. GAMMA SCALPING:
   • Scalp when gamma P&L > {best_trial.params['gamma_scalp_threshold']:.1%} of notional
   
4. EXPECTED COSTS:
   • ~{best_trial.user_attrs['hedge_count']} hedges per week
   • ~${best_trial.user_attrs['transaction_costs']:,.0f} in transaction costs/week
   
5. RISK PROFILE:
   • Expected weekly P&L: ${best_trial.user_attrs['net_pnl']:,.0f}
   • Max drawdown: ${abs(best_trial.user_attrs['max_drawdown']):,.0f}

⚠️  IMPORTANT: Re-run this optimization:
   - Monthly with fresh market data
   - After significant regime changes (vol crush/spike)
   - When changing position sizes significantly
""")

print("\n✅ Notebook complete!")